In [1]:
#@title Cell A — Install required libs (run once)
!pip install xgboost scikit-learn pandas tqdm joblib


In [2]:
#@title Cell A — Install required libs (run once)
!pip install xgboost scikit-learn pandas tqdm joblib

# ================================================
#  XGBOOST PRICE FORECASTING PIPELINE (VS CODE)
# ================================================

# === Imports ===
import os
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, timezone
from pymongo import MongoClient
from dotenv import load_dotenv
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from joblib import dump, load

# Reproducible randomness for model training & forecast generation
np.random.seed(42)
random.seed(42)

# ================================================
#  CELL B — CONNECT TO MONGODB AND LOAD DATA
# ================================================

load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")
if not MONGO_URI:
    raise ValueError("❌ MONGO_URI not found. Check your .env file.")

client = MongoClient(MONGO_URI)
db = client["ecom_tracker"]

scraped_coll = db["price_history"]
synthetic_coll = db["Sythetic data 2"]

scraped_docs = list(scraped_coll.find({}))
synthetic_docs = list(synthetic_coll.find({}))

print(f"Scraped docs: {len(scraped_docs)}")
print(f"Synthetic docs: {len(synthetic_docs)}")

df_scraped = pd.DataFrame(scraped_docs)
df_synth = pd.DataFrame(synthetic_docs)

for df_temp in (df_scraped, df_synth):
    if "scraped_at" in df_temp.columns:
        df_temp["scraped_at"] = pd.to_datetime(df_temp["scraped_at"], utc=True)
    for col in ["price", "original_price", "discount_percent"]:
        if col in df_temp.columns:
            df_temp[col] = pd.to_numeric(df_temp[col], errors="coerce")

df = pd.concat([df_scraped, df_synth], ignore_index=True, sort=False)
df = df[["asin", "price", "original_price", "discount_percent", "scraped_at"]]
df = df.dropna(subset=["asin", "price", "scraped_at"]).sort_values(["asin", "scraped_at"])
print(f"✅ Combined dataset loaded successfully! Rows: {len(df)}, ASINs: {df['asin'].nunique()}")




Scraped docs: 270
Synthetic docs: 1365
✅ Combined dataset loaded successfully! Rows: 1635, ASINs: 5
Scraped docs: 270
Synthetic docs: 1365
✅ Combined dataset loaded successfully! Rows: 1635, ASINs: 5


In [3]:
# --- Cell C: Feature Engineering (clean version, no warnings) ---
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
import random

# Define Amazon event windows in UTC
def ist_to_utc(year, month, day, hour=0, minute=0):
    return datetime(year, month, day, hour, minute,
                    tzinfo=timezone(timedelta(hours=5, minutes=30))
                    ).astimezone(timezone.utc)

# Amazon sale events (for 2025)
GIF_START = ist_to_utc(2025, 9, 23, 0, 0)
GIF_END   = ist_to_utc(2025, 10, 20, 23, 59)
PRIME_START = ist_to_utc(2025, 7, 12, 0, 0)
PRIME_END   = ist_to_utc(2025, 7, 14, 23, 59)

def is_gif(dt): return (dt >= GIF_START) and (dt <= GIF_END)
def is_prime(dt): return (dt >= PRIME_START) and (dt <= PRIME_END)

def make_features_for_group(g):
    g = g.sort_values("scraped_at").copy()
    g["scraped_at"] = pd.to_datetime(g["scraped_at"], utc=True)
    g = g.reset_index(drop=True)

    # Basic time features
    g["dayofweek"] = g["scraped_at"].dt.weekday
    g["day"] = g["scraped_at"].dt.day
    g["month"] = g["scraped_at"].dt.month
    g["hour"] = g["scraped_at"].dt.hour
    g["is_weekend"] = (g["dayofweek"] >= 5).astype(int)
    g["is_gif"] = g["scraped_at"].apply(lambda x: 1 if is_gif(x.to_pydatetime()) else 0)
    g["is_prime"] = g["scraped_at"].apply(lambda x: 1 if is_prime(x.to_pydatetime()) else 0)
    g["price_to_mrp"] = g["price"] / g["original_price"]

    # Lag features
    lag_steps = [1, 2, 3, 6, 9, 21, 42]
    for L in lag_steps:
        g[f"lag_price_{L}"] = g["price"].shift(L)
        g[f"lag_pct_{L}"] = g["price_to_mrp"].shift(L)

    # Rolling statistics
    g["roll_mean_3"] = g["price"].rolling(3, min_periods=1).mean().shift(1)
    g["roll_std_3"] = g["price"].rolling(3, min_periods=1).std().shift(1).fillna(0)
    g["roll_mean_9"] = g["price"].rolling(9, min_periods=1).mean().shift(1)
    g["roll_mean_27"] = g["price"].rolling(27, min_periods=1).mean().shift(1)
    g["pct_change_3"] = g["price"].pct_change(3).shift(1).fillna(0)

    # Fill missing values: prefer forward-fill to avoid leaking future values into past rows.
    # Use back-fill only to fill any initial rows if absolutely necessary.
    g = g.ffill().bfill()  # Prefer forward fill first to avoid leaking future data

    return g

# Apply feature generation per ASIN
groups = [make_features_for_group(g) for _, g in df.groupby("asin")]
df_feat = pd.concat(groups, ignore_index=True, sort=False)

print(f"✅ Feature dataframe ready! Shape: {df_feat.shape}")
print(df_feat.head())


✅ Feature dataframe ready! Shape: (1635, 32)
         asin    price  original_price  discount_percent  \
0  B09CYX92NB  9842.62          9999.0              1.56   
1  B09CYX92NB  9999.00          9999.0              0.00   
2  B09CYX92NB  9983.33          9999.0              0.16   
3  B09CYX92NB  9637.54          9999.0              3.61   
4  B09CYX92NB  9593.19          9999.0              4.06   

                 scraped_at  dayofweek  day  month  hour  is_weekend  ...  \
0 2025-08-04 09:49:00+00:00          0    4      8     9           0  ...   
1 2025-08-04 09:50:00+00:00          0    4      8     9           0  ...   
2 2025-08-04 09:51:00+00:00          0    4      8     9           0  ...   
3 2025-08-05 17:15:00+00:00          1    5      8    17           0  ...   
4 2025-08-05 17:16:00+00:00          1    5      8    17           0  ...   

   lag_pct_9  lag_price_21  lag_pct_21  lag_price_42  lag_pct_42  roll_mean_3  \
0    0.98436       9842.62     0.98436       9842.

In [4]:
# ================================================
#  CELL D — METRICS FUNCTION
# ================================================
def evaluate_preds(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, 1, y_true))) * 100
    return {"MAE": mae, "RMSE": rmse, "MAPE(%)": mape}



In [5]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from joblib import dump
import numpy as np

# Label encode ASINs
asin_le = LabelEncoder()
df_feat["asin_encoded"] = asin_le.fit_transform(df_feat["asin"])

# Features now include product code
exclude_cols = ["asin", "price", "original_price", "discount_percent", "scraped_at"]
feature_cols = [c for c in df_feat.columns if c not in exclude_cols]
print("Feature columns:", feature_cols)

# Prepare data
df_feat = df_feat.sort_values("scraped_at")
X = df_feat[feature_cols].values
y = df_feat["price"].values

# Time-series split (on overall time, not per ASIN)
tscv = TimeSeriesSplit(n_splits=5)
model_global = XGBRegressor(
    n_estimators=800,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=0.1,
    reg_lambda=2,
    objective="reg:squarederror",
    random_state=42,
)

# Train (optionally loop through CV manually to evaluate)
model_global.fit(X, y)

# Save model + metadata (feature columns and label encoder) together so it can be loaded reproducibly
from joblib import dump

dump({'model': model_global, 'feature_cols': feature_cols, 'asin_le': asin_le}, "model_bundle.joblib")
print("✅ Global XGBoost model trained and saved as 'model_bundle.joblib' (model + metadata)!")


Feature columns: ['dayofweek', 'day', 'month', 'hour', 'is_weekend', 'is_gif', 'is_prime', 'price_to_mrp', 'lag_price_1', 'lag_pct_1', 'lag_price_2', 'lag_pct_2', 'lag_price_3', 'lag_pct_3', 'lag_price_6', 'lag_pct_6', 'lag_price_9', 'lag_pct_9', 'lag_price_21', 'lag_pct_21', 'lag_price_42', 'lag_pct_42', 'roll_mean_3', 'roll_std_3', 'roll_mean_9', 'roll_mean_27', 'pct_change_3', 'asin_encoded']
✅ Global XGBoost model trained and saved as 'model_bundle.joblib' (model + metadata)!
✅ Global XGBoost model trained and saved as 'model_bundle.joblib' (model + metadata)!


In [7]:
# --- Cell F: 30-Day Forecast using Global Model ---
from joblib import load
import pandas as pd
import numpy as np
from datetime import timedelta

# Load trained model + label encoder (support bundle + legacy files)
try:
    bundle = load("model_bundle.joblib")
    if isinstance(bundle, dict):
        model_global = bundle.get("model", bundle)
        asin_le = bundle.get("asin_le", None)
        # if feature_cols are stored in the bundle, use them; otherwise keep existing
        try:
            feature_cols = bundle.get("feature_cols", feature_cols)
        except NameError:
            feature_cols = bundle.get("feature_cols")
    else:
        # legacy single-object file
        model_global = bundle
        try:
            asin_le = load("asin_labelencoder.joblib")
        except Exception:
            asin_le = None
    print("Loaded model from 'model_bundle.joblib'")
except FileNotFoundError:
    # Fallback for older notebooks / artifacts
    print("model_bundle.joblib not found, falling back to legacy files")
    model_global = load("xgb_global.joblib")
    try:
        asin_le = load("asin_labelencoder.joblib")
    except Exception:
        asin_le = None

forecast_rows = []

def generate_future_timestamps(last_ts, days=30, samples_per_day=3):
    """Generate timestamps for next 30 days with slight time variations."""
    future = []
    cur_date = (last_ts + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)

    for _ in range(days):
        base_hour = np.random.choice([9, 14, 20])  # 3 typical scraping times
        for i in range(samples_per_day):
            ts = cur_date.replace(hour=base_hour, minute=i)
            ts = pd.to_datetime(ts)
            if ts.tzinfo is None or ts.tzinfo.utcoffset(ts) is None:
                ts_utc = ts.tz_localize("UTC")
            else:
                ts_utc = ts.tz_convert("UTC")
            future.append(ts_utc)
        cur_date += timedelta(days=1)
    return future


# Forecast for each ASIN
for asin, g in df_feat.groupby("asin"):
    print(f"Forecasting ASIN: {asin}")
    if asin_le is None:
        raise RuntimeError("Label encoder (asin_le) not available. Ensure model_bundle.joblib contains 'asin_le' or provide 'asin_labelencoder.joblib'.")

    asin_code = asin_le.transform([asin])[0]
    g = g.sort_values("scraped_at").reset_index(drop=True)
    last_ts = g["scraped_at"].iloc[-1].to_pydatetime()

    future_ts = generate_future_timestamps(last_ts, 30, 3)
    rolling = g.copy()

    for ts in future_ts:
        new_row = {
            "asin": asin,
            "asin_encoded": asin_code,
            "scraped_at": ts,
            "original_price": rolling["original_price"].iloc[-1],
        }

        # Date/time features
        new_row["dayofweek"] = ts.weekday()
        new_row["day"], new_row["month"], new_row["hour"] = ts.day, ts.month, ts.hour
        new_row["is_weekend"] = 1 if ts.weekday() >= 5 else 0
        new_row["is_gif"], new_row["is_prime"] = int(is_gif(ts)), int(is_prime(ts))

        # Price relationships
        new_row["price_to_mrp"] = rolling["price"].iloc[-1] / new_row["original_price"]

        # Lag features
        for L in [1, 2, 3, 6, 9, 21, 42]:
            val = rolling["price"].iloc[-L] if len(rolling) >= L else rolling["price"].iloc[0]
            new_row[f"lag_price_{L}"] = val
            new_row[f"lag_pct_{L}"] = val / new_row["original_price"]

        # Rolling stats
        arr = rolling["price"].values
        new_row["roll_mean_3"]  = np.mean(arr[-3:])
        new_row["roll_mean_9"]  = np.mean(arr[-9:])
        new_row["roll_mean_27"] = np.mean(arr[-27:])
        new_row["roll_std_3"]   = np.std(arr[-3:])
        new_row["pct_change_3"] = (arr[-1] - arr[-4]) / arr[-4] if len(arr) > 4 else 0

        # Predict next price
        X_new = pd.DataFrame([new_row])[feature_cols].values
        pred = float(model_global.predict(X_new)[0])
        pred = max(0, min(pred, new_row["original_price"]))

        new_row["price"] = pred
        new_row["discount_percent"] = round((1 - pred / new_row["original_price"]) * 100, 2)

        forecast_rows.append({
            "asin": asin,
            "predicted_price": pred,
            "predicted_discount_percent": new_row["discount_percent"],
            "forecast_ts": ts
        })

        # Append prediction to maintain continuity
        rolling = pd.concat([rolling, pd.DataFrame([new_row])], ignore_index=True)

# Convert to DataFrame
df_forecast = pd.DataFrame(forecast_rows)
print(f"\n✅ Global model forecast complete! Rows: {len(df_forecast)}")
print(df_forecast.head())


Loaded model from 'model_bundle.joblib'
Forecasting ASIN: B09CYX92NB
Forecasting ASIN: B0C3HCD34R
Forecasting ASIN: B0C3HCD34R
Forecasting ASIN: B0DG2SLR9F
Forecasting ASIN: B0DG2SLR9F
Forecasting ASIN: B0DHKJ5HWL
Forecasting ASIN: B0DHKJ5HWL
Forecasting ASIN: B0DV5HX4JZ
Forecasting ASIN: B0DV5HX4JZ

✅ Global model forecast complete! Rows: 450
         asin  predicted_price  predicted_discount_percent  \
0  B09CYX92NB      5994.602539                       40.05   
1  B09CYX92NB      5994.561523                       40.05   
2  B09CYX92NB      5994.561523                       40.05   
3  B09CYX92NB      5993.985840                       40.05   
4  B09CYX92NB      5993.980469                       40.05   

                forecast_ts  
0 2025-11-07 20:00:00+00:00  
1 2025-11-07 20:01:00+00:00  
2 2025-11-07 20:02:00+00:00  
3 2025-11-08 09:00:00+00:00  
4 2025-11-08 09:01:00+00:00  

✅ Global model forecast complete! Rows: 450
         asin  predicted_price  predicted_discount_perce

In [ ]:
# # --- Optional: Save forecast to MongoDB ---
# forecast_coll = db["price_forecast_30d"]
# forecast_coll.insert_many(df_forecast.to_dict("records"))
# print(f"✅ Forecast saved to MongoDB collection 'price_forecast_30d'")

# # --- Optional: Save to CSV ---
# df_forecast.to_csv("30_day_forecast_global.csv", index=False)
# print("📁 Forecast also saved as '30_day_forecast_global.csv'")


In [9]:
# --- Cell H: Evaluation Summary (for Global Model) ---
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

def evaluate_global_model(df_feat, model):
    """Compute evaluation metrics overall and per ASIN."""
    X = df_feat[feature_cols].values
    y_true = df_feat["price"].values
    y_pred = model.predict(X)
    
    results = []
    for asin, g in df_feat.groupby("asin"):
        Xg, yg = g[feature_cols].values, g["price"].values
        yg_pred = model.predict(Xg)
        mae = mean_absolute_error(yg, yg_pred)
        rmse = np.sqrt(mean_squared_error(yg, yg_pred))
        denom = np.where(yg == 0, 1, yg)
        mape = np.mean(np.abs((yg - yg_pred) / denom)) * 100
        results.append((asin, mae, rmse, mape))
    
    df_eval = pd.DataFrame(results, columns=["ASIN", "MAE", "RMSE", "MAPE(%)"])
    df_eval = df_eval.sort_values("MAPE(%)")
    return df_eval

# Evaluate
# If model was saved as a bundle, load appropriately. For example:
# from joblib import load
# bundle = load("model_bundle.joblib")
# model_global = bundle['model']

try:
    df_eval = evaluate_global_model(df_feat, model_global)
    print("📊 Model Evaluation Summary (Global XGBoost):")
    print(df_eval.to_string(index=False))
    print("\n✅ Evaluation complete!")
except Exception as e:
    print("Evaluation failed:", e)


📊 Model Evaluation Summary (Global XGBoost):
      ASIN      MAE     RMSE  MAPE(%)
B09CYX92NB 0.120466 0.170969 0.001534
B0DV5HX4JZ 0.171329 0.256970 0.002784
B0C3HCD34R 0.175896 0.245648 0.003564
B0DHKJ5HWL 0.125020 0.175357 0.004533
B0DG2SLR9F 0.159918 0.230151 0.004982

✅ Evaluation complete!


In [10]:
# --- Cell I: Out-of-sample per-ASIN holdout evaluation (14 days) ---
# This cell performs a time-based holdout (last 14 days per ASIN), trains a global model on the remaining
# data, and evaluates model vs a naive baseline. Results are saved to `holdout_eval.csv`.

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from datetime import timedelta

holdout_days = 14
print(f"Running per-ASIN holdout evaluation with {holdout_days} days holdout...")

# Build test index: last `holdout_days` for each ASIN
test_idx = []
for asin, g in df_feat.groupby('asin'):
    last_ts = g['scraped_at'].max()
    cutoff = last_ts - timedelta(days=holdout_days)
    idx = g[g['scraped_at'] > cutoff].index.tolist()
    if len(idx) > 0:
        test_idx.extend(idx)

if len(test_idx) == 0:
    raise RuntimeError('No test rows found for chosen holdout_days. Increase dataset or reduce holdout_days.')

# Create train/test splits
df_test = df_feat.loc[sorted(test_idx)].copy()
df_train = df_feat.drop(index=test_idx).copy()

# Remove ASINs with no training data
asins_with_train = set(df_train['asin'].unique())
df_test = df_test[df_test['asin'].isin(asins_with_train)].copy()

print(f"Train rows: {len(df_train)}, Test rows: {len(df_test)}, ASINs in test after filtering: {df_test['asin'].nunique()}")

# Prepare label encoder and encoded column consistent with training
le = LabelEncoder()
le.fit(df_train['asin'])
df_train['asin_encoded'] = le.transform(df_train['asin'])
df_test['asin_encoded'] = le.transform(df_test['asin'])

# Feature columns (same exclude rule as training)
exclude_cols = ["asin", "price", "original_price", "discount_percent", "scraped_at"]
feature_cols_holdout = [c for c in df_train.columns if c not in exclude_cols]

# Train global model on df_train
X_train = df_train[feature_cols_holdout].values
y_train = df_train['price'].values

model_holdout = XGBRegressor(
    n_estimators=800,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=0.1,
    reg_lambda=2,
    objective='reg:squarederror',
    random_state=42,
)

print('Training global model on train split...')
model_holdout.fit(X_train, y_train)
print('Training complete.')

# Predict on test set
X_test = df_test[feature_cols_holdout].values
y_test = df_test['price'].values

y_pred = model_holdout.predict(X_test)

df_test = df_test.reset_index(drop=True)
df_test['pred'] = y_pred

# Naive baseline: last observed price from training for that ASIN
last_price_map = df_train.groupby('asin')['price'].last().to_dict()

def naively_predict(row):
    return last_price_map.get(row['asin'], df_train['price'].median())

df_test['naive_pred'] = df_test.apply(naively_predict, axis=1)

# Compute per-ASIN metrics
results = []
for asin, g in df_test.groupby('asin'):
    yg = g['price'].values
    yp = g['pred'].values
    yb = g['naive_pred'].values
    mae = mean_absolute_error(yg, yp)
    rmse = np.sqrt(mean_squared_error(yg, yp))
    denom = np.where(yg == 0, 1, yg)
    mape = np.mean(np.abs((yg - yp) / denom)) * 100
    wmape = np.sum(np.abs(yg - yp)) / np.sum(np.abs(yg)) if np.sum(np.abs(yg)) > 0 else np.nan

    # Baseline errors (naive)
    mae_base = mean_absolute_error(yg, yb)
    wmape_base = np.sum(np.abs(yg - yb)) / np.sum(np.abs(yg)) if np.sum(np.abs(yg)) > 0 else np.nan

    results.append({
        'ASIN': asin,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE(%)': mape,
        'WMAPE': wmape,
        'MAE_naive': mae_base,
        'WMAPE_naive': wmape_base,
        'n_test_rows': len(g)
    })

df_holdout_eval = pd.DataFrame(results).sort_values('WMAPE')

# Summary metrics
median_wmape = df_holdout_eval['WMAPE'].median()
q75_wmape = df_holdout_eval['WMAPE'].quantile(0.75)
frac_under_10 = (df_holdout_eval['WMAPE'] < 0.10).mean()
frac_under_20 = (df_holdout_eval['WMAPE'] < 0.20).mean()
frac_improved_over_naive = (df_holdout_eval['WMAPE'] < df_holdout_eval['WMAPE_naive']).mean()

print('\nHoldout summary:')
print(f"ASINs evaluated: {len(df_holdout_eval)}")
print(f"Median WMAPE: {median_wmape:.4f}")
print(f"75th percentile WMAPE: {q75_wmape:.4f}")
print(f"Fraction ASINs WMAPE <10%: {frac_under_10:.2%}")
print(f"Fraction ASINs WMAPE <20%: {frac_under_20:.2%}")
print(f"Fraction improved over naive baseline: {frac_improved_over_naive:.2%}")

# Save results
out_path = 'holdout_eval.csv'
df_holdout_eval.to_csv(out_path, index=False)
print(f"Saved per-ASIN holdout evaluation to {out_path}")

# Display top/bottom examples
print('\nTop 5 ASINs by best WMAPE:')
print(df_holdout_eval.head().to_string(index=False))
print('\nTop 5 ASINs by worst WMAPE:')
print(df_holdout_eval.tail().to_string(index=False))


Running per-ASIN holdout evaluation with 14 days holdout...
Train rows: 1200, Test rows: 435, ASINs in test after filtering: 5
Training global model on train split...
Training complete.

Holdout summary:
ASINs evaluated: 5
Median WMAPE: 0.0607
75th percentile WMAPE: 0.0798
Fraction ASINs WMAPE <10%: 80.00%
Fraction ASINs WMAPE <20%: 80.00%
Fraction improved over naive baseline: 100.00%
Saved per-ASIN holdout evaluation to holdout_eval.csv

Top 5 ASINs by best WMAPE:
      ASIN        MAE       RMSE   MAPE(%)    WMAPE   MAE_naive  WMAPE_naive  n_test_rows
B09CYX92NB 142.489406 169.413676  2.264362 0.019561 1982.235930     0.272126           86
B0C3HCD34R  95.875705 132.432817  2.121054 0.019992  761.457356     0.158776           87
B0DV5HX4JZ 295.909177 564.515459 10.379373 0.060744 3456.558046     0.709556           87
B0DG2SLR9F 224.533314 287.463910  9.758607 0.079788  890.413448     0.316411           87
B0DHKJ5HWL 773.720662 965.418119 58.983223 0.364244 1341.798068     0.631678   